<a href="https://colab.research.google.com/github/Sudin2001/Project-Of-ML/blob/main/Spare_parts_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Step 1: Import Required Libraries and Load Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

print("📊 EXCAVATOR SUPPLIER RISK ANALYSIS PROJECT")
print("=" * 50)
print("\n🔄 Step 1: Loading Data Files...")

# Load all the datasets
try:
    parts_df = pd.read_csv('/content/drive/MyDrive/Risk model on spare parts supply chain disruption/excavator_spare_parts_master.csv')
    suppliers_df = pd.read_csv('/content/drive/MyDrive/Risk model on spare parts supply chain disruption/suppliers_master.csv')
    supplier_parts_df = pd.read_csv('/content/drive/MyDrive/Risk model on spare parts supply chain disruption/supplier_parts_mapping.csv')
    po_df = pd.read_csv('/content/drive/MyDrive/Risk model on spare parts supply chain disruption/purchase_orders.csv')
    production_df = pd.read_csv('/content/drive/MyDrive/Risk model on spare parts supply chain disruption/production_schedule.csv')
    parts_req_df = pd.read_csv('/content/drive/MyDrive/Risk model on spare parts supply chain disruption/parts_requirements.csv')
    inventory_df = pd.read_csv('/content/drive/MyDrive/Risk model on spare parts supply chain disruption/inventory_status.csv')
    risk_events_df = pd.read_csv('/content/drive/MyDrive/Risk model on spare parts supply chain disruption/risk_events.csv')
    supplier_risk_df = pd.read_csv('/content/drive/MyDrive/Risk model on spare parts supply chain disruption/supplier_risk_analysis.csv')

    print("✅ All datasets loaded successfully!")
    print(f"   • Parts Master: {len(parts_df)} records")
    print(f"   • Suppliers: {len(suppliers_df)} records")
    print(f"   • Purchase Orders: {len(po_df)} records")
    print(f"   • Risk Events: {len(risk_events_df)} records")

except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("Please ensure all CSV files are in the current directory")

📊 EXCAVATOR SUPPLIER RISK ANALYSIS PROJECT

🔄 Step 1: Loading Data Files...
✅ All datasets loaded successfully!
   • Parts Master: 32 records
   • Suppliers: 15 records
   • Purchase Orders: 1654 records
   • Risk Events: 945 records


In [ ]:
# Step 3: Feature Engineering for Risk Analysis
print("\n🔧 Step 3: Feature Engineering for Risk Analysis")
print("=" * 50)

print("\n📈 Why Feature Engineering?")
print("Feature engineering transforms raw data into meaningful predictors for risk models.")
print("For supplier risk analysis, we need features that capture:")
print("- Historical performance patterns")
print("- Supplier characteristics and reliability")
print("- Order complexity and urgency")
print("- Seasonal and temporal patterns")

# Convert date columns to datetime
po_df['PO_Date'] = pd.to_datetime(po_df['PO_Date'])
po_df['Promised_Delivery_Date'] = pd.to_datetime(po_df['Promised_Delivery_Date'])
po_df['Actual_Delivery_Date'] = pd.to_datetime(po_df['Actual_Delivery_Date'])

# Create enhanced dataset by merging with master data
print("\n🔗 Creating Enhanced Dataset...")

# Merge with parts and suppliers data
enhanced_df = po_df.merge(parts_df[['Part_ID', 'Category', 'Criticality']], on='Part_ID')
enhanced_df = enhanced_df.merge(suppliers_df[['Supplier_ID', 'Supplier_Type', 'Years_Partnership',
                                            'Financial_Rating', 'Historical_OTD_Pct', 'Location']], on='Supplier_ID')

print(f"✅ Enhanced dataset created with {enhanced_df.shape[1]} features")

# Feature Engineering: Historical Performance Features
print("\n📊 Creating Historical Performance Features...")

# 1. Supplier Rolling Performance (Last 30 days, 60 days, 90 days)
def calculate_rolling_metrics(df, supplier_id, reference_date, window_days):
    """Calculate supplier performance over a rolling window"""
    cutoff_date = reference_date - pd.Timedelta(days=window_days)
    historical_pos = df[(df['Supplier_ID'] == supplier_id) &
                       (df['PO_Date'] >= cutoff_date) &
                       (df['PO_Date'] < reference_date)]

    if len(historical_pos) == 0:
        return {
            'orders_count': 0,
            'otd_rate': 0.5,  # Default neutral value
            'avg_delay': 0,
            'delay_variance': 0,
            'total_value': 0
        }

    return {
        'orders_count': len(historical_pos),
        'otd_rate': historical_pos['Is_On_Time'].mean(),
        'avg_delay': historical_pos[historical_pos['Is_Late']]['Delay_Days'].mean() if sum(historical_pos['Is_Late']) > 0 else 0,
        'delay_variance': historical_pos['Delay_Days'].var(),
        'total_value': historical_pos['Order_Value_INR'].sum()
    }

# Calculate rolling metrics for each PO (this simulates real-time risk scoring)
rolling_features = []

for idx, row in enhanced_df.iterrows():
    # Calculate 30-day and 90-day rolling metrics
    metrics_30 = calculate_rolling_metrics(enhanced_df, row['Supplier_ID'], row['PO_Date'], 30)
    metrics_90 = calculate_rolling_metrics(enhanced_df, row['Supplier_ID'], row['PO_Date'], 90)

    rolling_features.append({
        'PO_Number': row['PO_Number'],
        'Supplier_30d_OTD': metrics_30['otd_rate'],
        'Supplier_30d_Orders': metrics_30['orders_count'],
        'Supplier_30d_AvgDelay': metrics_30['avg_delay'] if not pd.isna(metrics_30['avg_delay']) else 0,
        'Supplier_90d_OTD': metrics_90['otd_rate'],
        'Supplier_90d_Orders': metrics_90['orders_count'],
        'Supplier_90d_DelayVar': metrics_90['delay_variance'] if not pd.isna(metrics_90['delay_variance']) else 0,
    })

rolling_df = pd.DataFrame(rolling_features)
enhanced_df = enhanced_df.merge(rolling_df, on='PO_Number')

print("✅ Rolling performance features created")
print(f"   • 30-day supplier OTD rates calculated")
print(f"   • 90-day supplier performance variance calculated")

# Feature Engineering: Temporal and Contextual Features
print("\n🕒 Creating Temporal and Contextual Features...")

# Extract temporal features
enhanced_df['PO_Month'] = enhanced_df['PO_Date'].dt.month
enhanced_df['PO_Quarter'] = enhanced_df['PO_Date'].dt.quarter
enhanced_df['PO_DayOfWeek'] = enhanced_df['PO_Date'].dt.dayofweek
enhanced_df['PO_WeekOfYear'] = enhanced_df['PO_Date'].dt.isocalendar().week

# Seasonal indicators (construction industry patterns)
enhanced_df['Is_Peak_Season'] = enhanced_df['PO_Month'].isin([3, 4, 5, 9, 10, 11])
enhanced_df['Is_Low_Season'] = enhanced_df['PO_Month'].isin([12, 1, 2])

# Order complexity features
enhanced_df['Order_Size_Category'] = pd.cut(enhanced_df['Quantity_Ordered'],
                                          bins=[0, 2, 5, 10, float('inf')],
                                          labels=['Small', 'Medium', 'Large', 'XLarge'])

enhanced_df['Order_Value_Category'] = pd.cut(enhanced_df['Order_Value_INR'],
                                           bins=[0, 100000, 500000, 1000000, float('inf')],
                                           labels=['Low', 'Medium', 'High', 'Premium'])

# Part and supplier risk features
criticality_risk = {'A': 3, 'B': 2, 'C': 1}
enhanced_df['Part_Risk_Score'] = enhanced_df['Criticality'].map(criticality_risk)

rating_risk = {'AAA': 1, 'AA+': 2, 'AA': 3, 'AA-': 4, 'A+': 5, 'A': 6}
enhanced_df['Financial_Risk_Score'] = enhanced_df['Financial_Rating'].map(rating_risk)

# Lead time risk indicator
enhanced_df['Lead_Time_Risk'] = (enhanced_df['Promised_Lead_Time'] > enhanced_df['Promised_Lead_Time'].median()).astype(int)

print("✅ Temporal and contextual features created")
print(f"   • Seasonal patterns identified")
print(f"   • Order complexity categorized")
print(f"   • Risk scores calculated")

# Display sample of enhanced features
print("\n📋 Sample Enhanced Features:")
feature_cols = ['PO_Number', 'Supplier_ID', 'Criticality', 'Supplier_30d_OTD', 'Part_Risk_Score',
                'Is_Peak_Season', 'Order_Size_Category', 'Delay_Days', 'Is_Late']
print(enhanced_df[feature_cols].head(8))


🔧 Step 3: Feature Engineering for Risk Analysis

📈 Why Feature Engineering?
Feature engineering transforms raw data into meaningful predictors for risk models.
For supplier risk analysis, we need features that capture:
- Historical performance patterns
- Supplier characteristics and reliability
- Order complexity and urgency
- Seasonal and temporal patterns

🔗 Creating Enhanced Dataset...
✅ Enhanced dataset created with 20 features

📊 Creating Historical Performance Features...
✅ Rolling performance features created
   • 30-day supplier OTD rates calculated
   • 90-day supplier performance variance calculated

🕒 Creating Temporal and Contextual Features...
✅ Temporal and contextual features created
   • Seasonal patterns identified
   • Order complexity categorized
   • Risk scores calculated

📋 Sample Enhanced Features:
  PO_Number Supplier_ID Criticality  Supplier_30d_OTD  Part_Risk_Score  \
0  PO001000      SUP105           C               0.5                1   
1  PO001001      S

In [ ]:
# Step 4: Predictive Modeling for Delivery Risk
print("\n🤖 Step 4: Predictive Modeling for Delivery Risk")
print("=" * 50)

print("\n🎯 Why Predictive Modeling?")
print("Predictive models help us:")
print("- Identify high-risk purchase orders BEFORE delays occur")
print("- Quantify the probability of late delivery")
print("- Understand which factors drive delivery risks")
print("- Enable proactive risk mitigation")

# Prepare data for modeling
print("\n🔧 Preparing Data for Machine Learning...")

# Select features for the model
model_features = [
    'Promised_Lead_Time', 'Quantity_Ordered', 'Order_Value_INR',
    'Part_Risk_Score', 'Financial_Risk_Score', 'Years_Partnership',
    'Supplier_30d_OTD', 'Supplier_90d_OTD', 'Supplier_90d_DelayVar',
    'Is_Peak_Season', 'PO_Month', 'PO_DayOfWeek', 'Lead_Time_Risk'
]

# Create feature matrix
X = enhanced_df[model_features].copy()

# Handle categorical variables
X['Is_Peak_Season'] = X['Is_Peak_Season'].astype(int)
X['Lead_Time_Risk'] = X['Lead_Time_Risk'].astype(int)

# Fill any missing values
X = X.fillna(0)

# Target variable: Late delivery (binary)
y_binary = enhanced_df['Is_Late'].astype(int)

# Target variable: Delay days (regression)
y_delay = enhanced_df['Delay_Days'].copy()

print(f"✅ Feature matrix prepared: {X.shape}")
print(f"   • Features: {len(model_features)} predictive variables")
print(f"   • Target distribution - Late deliveries: {y_binary.mean():.1%}")

# Split data for training and testing
X_train, X_test, y_train_binary, y_test_binary = train_test_split(
    X, y_binary, test_size=0.3, random_state=42, stratify=y_binary
)

X_train_delay, X_test_delay, y_train_delay, y_test_delay = train_test_split(
    X, y_delay, test_size=0.3, random_state=42
)

print(f"✅ Data split - Training: {X_train.shape[0]}, Testing: {X_test.shape[0]}")

# Model 1: Binary Classification - Will it be late?
print("\n🎲 Building Binary Classification Model (Late vs On-Time)...")

# Random Forest Classifier
rf_classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight='balanced'  # Handle class imbalance
)

rf_classifier.fit(X_train, y_train_binary)

# Predictions
y_pred_binary = rf_classifier.predict(X_test)
y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]

# Model performance
auc_score = roc_auc_score(y_test_binary, y_pred_proba)
print(f"✅ Binary Classification Model Performance:")
print(f"   • AUC Score: {auc_score:.3f}")
print(f"   • Accuracy: {(y_pred_binary == y_test_binary).mean():.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': model_features,
    'Importance': rf_classifier.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n📊 Top 5 Most Important Features for Delay Prediction:")
for idx, row in feature_importance.head(5).iterrows():
    print(f"   {idx+1}. {row['Feature']}: {row['Importance']:.3f}")

# Model 2: Logistic Regression for Interpretability
print("\n📈 Building Logistic Regression Model (for interpretability)...")

# Scale features for logistic regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_classifier = LogisticRegression(random_state=42, class_weight='balanced')
lr_classifier.fit(X_train_scaled, y_train_binary)

# Logistic regression predictions
y_pred_lr = lr_classifier.predict(X_test_scaled)
y_pred_lr_proba = lr_classifier.predict_proba(X_test_scaled)[:, 1]

auc_lr = roc_auc_score(y_test_binary, y_pred_lr_proba)
print(f"✅ Logistic Regression Performance:")
print(f"   • AUC Score: {auc_lr:.3f}")
print(f"   • Accuracy: {(y_pred_lr == y_test_binary).mean():.3f}")

# Extract coefficients (risk factors)
coefficients = pd.DataFrame({
    'Feature': model_features,
    'Coefficient': lr_classifier.coef_[0],
    'Risk_Impact': ['Increases Risk' if x > 0 else 'Reduces Risk' for x in lr_classifier.coef_[0]]
}).sort_values('Coefficient', ascending=False, key=abs)

print(f"\n📊 Top Risk Factors (Logistic Regression Coefficients):")
for idx, row in coefficients.head(5).iterrows():
    print(f"   • {row['Feature']}: {row['Coefficient']:.3f} ({row['Risk_Impact']})")

# Save model predictions for further analysis
enhanced_df['Delay_Probability_RF'] = 0.0
enhanced_df['Delay_Probability_LR'] = 0.0

# Predict on full dataset
enhanced_df.loc[X_test.index, 'Delay_Probability_RF'] = y_pred_proba
enhanced_df.loc[X_test.index, 'Delay_Probability_LR'] = y_pred_lr_proba

print("\n✅ Predictive models completed and saved to dataset")


🤖 Step 4: Predictive Modeling for Delivery Risk

🎯 Why Predictive Modeling?
Predictive models help us:
- Identify high-risk purchase orders BEFORE delays occur
- Quantify the probability of late delivery
- Understand which factors drive delivery risks
- Enable proactive risk mitigation

🔧 Preparing Data for Machine Learning...
✅ Feature matrix prepared: (1654, 13)
   • Features: 13 predictive variables
   • Target distribution - Late deliveries: 22.3%
✅ Data split - Training: 1157, Testing: 497

🎲 Building Binary Classification Model (Late vs On-Time)...
✅ Binary Classification Model Performance:
   • AUC Score: 0.620
   • Accuracy: 0.767

📊 Top 5 Most Important Features for Delay Prediction:
   9. Supplier_90d_DelayVar: 0.169
   8. Supplier_90d_OTD: 0.149
   3. Order_Value_INR: 0.139
   7. Supplier_30d_OTD: 0.093
   12. PO_DayOfWeek: 0.085

📈 Building Logistic Regression Model (for interpretability)...
✅ Logistic Regression Performance:
   • AUC Score: 0.610
   • Accuracy: 0.579

📊 T

In [ ]:
# Step 5: Comprehensive Risk Scoring System
print("\n🎯 Step 5: Comprehensive Risk Scoring System")
print("=" * 50)

print("\n🔍 Why Risk Scoring?")
print("Risk scoring provides:")
print("- Single composite metric combining multiple risk factors")
print("- Supplier ranking and benchmarking capability")
print("- Clear categorization for decision making")
print("- Tracking risk evolution over time")

# Calculate comprehensive supplier risk scores
print("\n📊 Calculating Supplier Risk Scores...")

def calculate_supplier_risk_score(supplier_id, df):
    """Calculate comprehensive risk score for a supplier"""
    supplier_data = df[df['Supplier_ID'] == supplier_id]

    if len(supplier_data) == 0:
        return {
            'Supplier_ID': supplier_id,
            'Risk_Score': 50,  # Neutral score
            'Risk_Category': 'Unknown'
        }

    # Performance metrics (40% weight)
    otd_performance = supplier_data['Is_On_Time'].mean()
    avg_delay = supplier_data[supplier_data['Is_Late']]['Delay_Days'].mean()
    avg_delay = avg_delay if not pd.isna(avg_delay) else 0

    performance_score = (1 - otd_performance) * 40 + min(avg_delay * 2, 20)

    # Volatility metrics (25% weight)
    delay_std = supplier_data['Delay_Days'].std()
    delay_std = delay_std if not pd.isna(delay_std) else 0
    volatility_score = min(delay_std * 2, 25)

    # Financial/Strategic metrics (20% weight)
    financial_risk = supplier_data['Financial_Risk_Score'].iloc[0]
    partnership_years = supplier_data['Years_Partnership'].iloc[0]

    strategic_score = financial_risk * 2 - (partnership_years / 15) * 10
    strategic_score = max(0, min(strategic_score, 20))

    # Predictive metrics (15% weight)
    avg_delay_prob = supplier_data['Delay_Probability_RF'].mean()
    predictive_score = avg_delay_prob * 15

    # Composite risk score (0-100 scale)
    total_risk_score = performance_score + volatility_score + strategic_score + predictive_score

    # Risk categorization
    if total_risk_score <= 25:
        risk_category = 'Low Risk'
    elif total_risk_score <= 50:
        risk_category = 'Medium Risk'
    elif total_risk_score <= 75:
        risk_category = 'High Risk'
    else:
        risk_category = 'Critical Risk'

    return {
        'Supplier_ID': supplier_id,
        'Total_Orders': len(supplier_data),
        'OTD_Performance': otd_performance,
        'Avg_Delay_Days': avg_delay,
        'Delay_Volatility': delay_std,
        'Avg_Delay_Probability': avg_delay_prob,
        'Total_Order_Value': supplier_data['Order_Value_INR'].sum(),
        'Performance_Score': performance_score,
        'Volatility_Score': volatility_score,
        'Strategic_Score': strategic_score,
        'Predictive_Score': predictive_score,
        'Risk_Score': total_risk_score,
        'Risk_Category': risk_category
    }

# Calculate risk scores for all suppliers
supplier_risk_analysis = []
for supplier_id in suppliers_df['Supplier_ID'].unique():
    risk_data = calculate_supplier_risk_score(supplier_id, enhanced_df)
    supplier_risk_analysis.append(risk_data)

supplier_risk_final_df = pd.DataFrame(supplier_risk_analysis)
supplier_risk_final_df = supplier_risk_final_df.merge(
    suppliers_df[['Supplier_ID', 'Supplier_Name', 'Location', 'Supplier_Type']],
    on='Supplier_ID'
)

print("✅ Supplier risk scores calculated")

# Display supplier risk ranking
print(f"\n🏆 Supplier Risk Ranking (Top 5 by Risk Score):")
risk_ranking = supplier_risk_final_df.sort_values('Risk_Score', ascending=False)
for idx, row in risk_ranking.head().iterrows():
    print(f"   {idx+1}. {row['Supplier_Name']} ({row['Supplier_ID']})")
    print(f"      Risk Score: {row['Risk_Score']:.1f} | Category: {row['Risk_Category']}")
    print(f"      OTD: {row['OTD_Performance']:.1%} | Avg Delay: {row['Avg_Delay_Days']:.1f} days")

# Step 6: Part-Level Risk Analysis
print("\n🔧 Step 6: Part-Level Risk Analysis")
print("=" * 40)

print("\n⚙️ Why Part-Level Analysis?")
print("Part analysis identifies:")
print("- Critical components with supply risks")
print("- Parts requiring safety stock adjustments")
print("- Components needing supplier diversification")

def calculate_part_risk_metrics(part_id, df):
    """Calculate risk metrics for a specific part"""
    part_data = df[df['Part_ID'] == part_id]

    if len(part_data) == 0:
        return None

    # Basic metrics
    total_orders = len(part_data)
    otd_performance = part_data['Is_On_Time'].mean()
    avg_delay = part_data[part_data['Is_Late']]['Delay_Days'].mean()
    avg_delay = avg_delay if not pd.isna(avg_delay) else 0

    # Supplier diversity
    supplier_count = part_data['Supplier_ID'].nunique()
    primary_supplier_share = part_data.groupby('Supplier_ID').size().max() / total_orders

    # Financial exposure
    total_order_value = part_data['Order_Value_INR'].sum()
    avg_order_value = part_data['Order_Value_INR'].mean()

    # Risk concentration
    criticality = part_data['Criticality'].iloc[0]
    criticality_multiplier = {'A': 3, 'B': 2, 'C': 1}[criticality]

    # Composite risk score for parts
    supply_risk = (1 - otd_performance) * 30
    concentration_risk = primary_supplier_share * 20
    criticality_risk = criticality_multiplier * 10
    delay_risk = min(avg_delay * 3, 20)

    part_risk_score = supply_risk + concentration_risk + criticality_risk + delay_risk

    return {
        'Part_ID': part_id,
        'Part_Name': parts_df[parts_df['Part_ID'] == part_id]['Part_Name'].iloc[0],
        'Category': part_data['Category'].iloc[0],
        'Criticality': criticality,
        'Total_Orders': total_orders,
        'Supplier_Count': supplier_count,
        'Primary_Supplier_Share': primary_supplier_share,
        'OTD_Performance': otd_performance,
        'Avg_Delay_Days': avg_delay,
        'Total_Order_Value': total_order_value,
        'Avg_Order_Value': avg_order_value,
        'Part_Risk_Score': part_risk_score,
        'Risk_Category': 'High Risk' if part_risk_score > 50 else 'Medium Risk' if part_risk_score > 25 else 'Low Risk'
    }

# Calculate part risk metrics
part_risk_analysis = []
for part_id in parts_df['Part_ID'].unique():
    part_risk_data = calculate_part_risk_metrics(part_id, enhanced_df)
    if part_risk_data:
        part_risk_analysis.append(part_risk_data)

part_risk_df = pd.DataFrame(part_risk_analysis)

print("✅ Part-level risk analysis completed")

# Display high-risk parts
print(f"\n⚠️ Top 5 Highest Risk Parts:")
high_risk_parts = part_risk_df.sort_values('Part_Risk_Score', ascending=False)
for idx, row in high_risk_parts.head().iterrows():
    print(f"   {idx+1}. {row['Part_Name']} ({row['Part_ID']})")
    print(f"      Risk Score: {row['Part_Risk_Score']:.1f} | Criticality: {row['Criticality']}")
    print(f"      Suppliers: {row['Supplier_Count']} | OTD: {row['OTD_Performance']:.1%}")

print(f"\n📊 Part Risk Distribution:")
print(part_risk_df['Risk_Category'].value_counts())

print(f"\n💰 Financial Exposure by Risk Category:")
financial_by_risk = part_risk_df.groupby('Risk_Category')['Total_Order_Value'].sum()
for category, value in financial_by_risk.items():
    print(f"   • {category}: ₹{value:,.0f}")


🎯 Step 5: Comprehensive Risk Scoring System

🔍 Why Risk Scoring?
Risk scoring provides:
- Single composite metric combining multiple risk factors
- Supplier ranking and benchmarking capability
- Clear categorization for decision making
- Tracking risk evolution over time

📊 Calculating Supplier Risk Scores...
✅ Supplier risk scores calculated

🏆 Supplier Risk Ranking (Top 5 by Risk Score):
   15. Bajaj Auto Ltd (SUP114)
      Risk Score: 45.4 | Category: Medium Risk
      OTD: 54.4% | Avg Delay: 6.0 days
   7. Action Construction Equipment (SUP106)
      Risk Score: 38.4 | Category: Medium Risk
      OTD: 50.6% | Avg Delay: 3.9 days
   8. Escorts Construction Equipment (SUP107)
      Risk Score: 32.0 | Category: Medium Risk
      OTD: 76.4% | Avg Delay: 4.5 days
   9. Volvo Construction Equipment (SUP108)
      Risk Score: 29.6 | Category: Medium Risk
      OTD: 72.9% | Avg Delay: 4.4 days
   6. Mahindra Construction Equipment (SUP105)
      Risk Score: 29.0 | Category: Medium Risk
  

In [ ]:
# Step 7: Monte Carlo Simulation for Fill Rate Impact
print("\n🎲 Step 7: Monte Carlo Simulation for Fill Rate Impact")
print("=" * 55)

print("\n🎯 Why Monte Carlo Simulation?")
print("Monte Carlo simulation helps us:")
print("- Model uncertainty in delivery times and demand")
print("- Estimate probability of stockouts and production delays")
print("- Quantify financial impact under different scenarios")
print("- Test effectiveness of risk mitigation strategies")

# Generate production requirements (simplified for simulation)
np.random.seed(42)

# Create a simplified production schedule
production_scenarios = []
simulation_start = pd.Timestamp('2025-02-01')

for week in range(12):  # 12-week simulation
    week_start = simulation_start + pd.Timedelta(weeks=week)

    # Weekly production requirements (realistic for excavator manufacturing)
    weekly_production = {
        'EX200': np.random.poisson(6),  # 6 units average per week
        'EX350': np.random.poisson(4),  # 4 units average per week
        'EX470': np.random.poisson(3),  # 3 units average per week
        'EX700': np.random.poisson(2)   # 2 units average per week
    }

    production_scenarios.append({
        'Week': week + 1,
        'Week_Start': week_start,
        'EX200_Units': weekly_production['EX200'],
        'EX350_Units': weekly_production['EX350'],
        'EX470_Units': weekly_production['EX470'],
        'EX700_Units': weekly_production['EX700'],
        'Total_Units': sum(weekly_production.values())
    })

production_forecast_df = pd.DataFrame(production_scenarios)
print(f"✅ Production forecast created: {len(production_forecast_df)} weeks")

# Define parts requirements per excavator model (Bill of Materials)
bom_requirements = {
    'EX200': {'PT1000': 1, 'PT1001': 2, 'PT1002': 1, 'PT1003': 2, 'PT1004': 1},
    'EX350': {'PT1000': 1, 'PT1001': 2, 'PT1002': 2, 'PT1003': 3, 'PT1004': 2, 'PT1005': 1},
    'EX700': {'PT1000': 2, 'PT1001': 3, 'PT1002': 2, 'PT1003': 4, 'PT1004': 2, 'PT1005': 2, 'PT1006': 1}
}

def simulate_delivery_scenario(supplier_id, part_id, order_date, num_simulations=1000):
    """
    Simulate delivery performance for a supplier-part combination
    Returns distribution of actual delivery dates
    """
    # Get supplier characteristics
    supplier_info = suppliers_df[suppliers_df['Supplier_ID'] == supplier_id].iloc[0]

    # Get historical performance for this supplier
    supplier_pos = enhanced_df[enhanced_df['Supplier_ID'] == supplier_id]

    if len(supplier_pos) > 0:
        # Use historical delay distribution
        historical_delays = supplier_pos['Delay_Days'].values
        otd_rate = supplier_pos['Is_On_Time'].mean()
    else:
        # Use supplier baseline performance
        otd_rate = supplier_info['Historical_OTD_Pct']
        historical_delays = np.random.normal(0, supplier_info['Lead_Time_Variance'], 100)

    # Simulate delivery delays
    simulated_delays = []
    base_lead_time = supplier_info['Avg_Lead_Time_Days']

    for _ in range(num_simulations):
        if np.random.random() < otd_rate:
            # On-time delivery (can be early)
            delay = np.random.choice([-2, -1, 0], p=[0.2, 0.3, 0.5])
        else:
            # Late delivery - sample from historical delays
            if len(historical_delays[historical_delays > 0]) > 0:
                delay = np.random.choice(historical_delays[historical_delays > 0])
            else:
                delay = np.random.randint(1, 10)

        # Add lead time variance
        variance = np.random.normal(0, supplier_info['Lead_Time_Variance'])
        total_delay = delay + variance

        delivery_date = order_date + pd.Timedelta(days=base_lead_time + total_delay)
        simulated_delays.append(total_delay)

    return simulated_delays

print("\n🎲 Running Monte Carlo Simulation...")

# Simulation parameters
num_simulations = 500
simulation_results = []

# For each week of production, simulate parts delivery
for _, week_row in production_forecast_df.head(8).iterrows():  # Simulate first 8 weeks
    week_start = week_row['Week_Start']
    week_num = week_row['Week']

    # Calculate parts needed for this week's production
    parts_needed = {}

    # EX200 requirements
    for part_id, qty_per_unit in bom_requirements['EX200'].items():
        parts_needed[part_id] = parts_needed.get(part_id, 0) + (week_row['EX200_Units'] * qty_per_unit)

    # EX350 requirements (if defined in BOM)
    if week_row['EX350_Units'] > 0:
        for part_id, qty_per_unit in bom_requirements['EX350'].items():
            parts_needed[part_id] = parts_needed.get(part_id, 0) + (week_row['EX350_Units'] * qty_per_unit)

    # Order date (parts ordered 3 weeks before production)
    order_date = week_start - pd.Timedelta(weeks=3)

    # Simulate delivery for each required part
    week_simulation_results = []

    for part_id, quantity_needed in parts_needed.items():
        # Get primary supplier for this part (simplified)
        primary_supplier = enhanced_df[enhanced_df['Part_ID'] == part_id]['Supplier_ID'].iloc[0]

        # Simulate delivery delays
        delivery_delays = simulate_delivery_scenario(primary_supplier, part_id, order_date, num_simulations)

        # Calculate delivery date distribution
        delivery_dates = [order_date + pd.Timedelta(days=22 + delay) for delay in delivery_delays]

        # Check if parts arrive on time for production (before week_start)
        on_time_deliveries = sum([1 for date in delivery_dates if date <= week_start])
        on_time_rate = on_time_deliveries / num_simulations

        # Average delay from required date
        avg_delay_from_required = np.mean([(date - week_start).days for date in delivery_dates])

        week_simulation_results.append({
            'Week': week_num,
            'Week_Start': week_start,
            'Part_ID': part_id,
            'Supplier_ID': primary_supplier,
            'Quantity_Needed': quantity_needed,
            'On_Time_Rate': on_time_rate,
            'Avg_Delay_From_Required': avg_delay_from_required,
            'Stockout_Risk': 1 - on_time_rate
        })

    simulation_results.extend(week_simulation_results)

    print(f"   ✅ Week {week_num} simulation completed - {len(parts_needed)} parts analyzed")

simulation_results_df = pd.DataFrame(simulation_results)

print(f"✅ Monte Carlo simulation completed")
print(f"   • Simulated {num_simulations} scenarios per part-week combination")
print(f"   • Analyzed {len(simulation_results)} part-week requirements")

# Calculate fill rate impact
print(f"\n📊 Fill Rate Impact Analysis:")

# Overall metrics
avg_on_time_rate = simulation_results_df['On_Time_Rate'].mean()
high_risk_parts = len(simulation_results_df[simulation_results_df['Stockout_Risk'] > 0.3])
total_part_requirements = len(simulation_results_df)

print(f"   • Average On-Time Rate: {avg_on_time_rate:.1%}")
print(f"   • High Stockout Risk (>30%): {high_risk_parts}/{total_part_requirements} requirements")
print(f"   • Average Delay from Required Date: {simulation_results_df['Avg_Delay_From_Required'].mean():.1f} days")

# Week-by-week fill rate
weekly_fill_rate = simulation_results_df.groupby('Week').agg({
    'On_Time_Rate': 'mean',
    'Stockout_Risk': 'max',
    'Quantity_Needed': 'sum'
}).round(3)

print(f"\n📅 Weekly Fill Rate Forecast:")
print(weekly_fill_rate.head(8))


🎲 Step 7: Monte Carlo Simulation for Fill Rate Impact

🎯 Why Monte Carlo Simulation?
Monte Carlo simulation helps us:
- Model uncertainty in delivery times and demand
- Estimate probability of stockouts and production delays
- Quantify financial impact under different scenarios
- Test effectiveness of risk mitigation strategies
✅ Production forecast created: 12 weeks

🎲 Running Monte Carlo Simulation...
   ✅ Week 1 simulation completed - 6 parts analyzed
   ✅ Week 2 simulation completed - 6 parts analyzed
   ✅ Week 3 simulation completed - 6 parts analyzed
   ✅ Week 4 simulation completed - 6 parts analyzed
   ✅ Week 5 simulation completed - 5 parts analyzed
   ✅ Week 6 simulation completed - 6 parts analyzed
   ✅ Week 7 simulation completed - 6 parts analyzed
   ✅ Week 8 simulation completed - 6 parts analyzed
✅ Monte Carlo simulation completed
   • Simulated 500 scenarios per part-week combination
   • Analyzed 47 part-week requirements

📊 Fill Rate Impact Analysis:
   • Average On-

In [ ]:
# Step 8: Optimization Recommendations and Business Intelligence Outputs
print("\n🎯 Step 8: Optimization Recommendations & BI Outputs")
print("=" * 55)

print("\n💡 Why Optimization Recommendations?")
print("Optimization provides:")
print("- Actionable strategies to reduce supply risk")
print("- Cost-benefit analysis of mitigation options")
print("- Prioritized list of interventions")
print("- ROI estimates for risk reduction initiatives")

# Safety Stock Optimization
print("\n📦 Safety Stock Optimization Analysis...")

def calculate_optimal_safety_stock(part_id, current_stock=None):
    """Calculate optimal safety stock based on demand variability and supplier reliability"""

    # Get part characteristics
    part_info = parts_df[parts_df['Part_ID'] == part_id].iloc[0]
    part_orders = enhanced_df[enhanced_df['Part_ID'] == part_id]

    if len(part_orders) == 0:
        return None

    # Calculate demand characteristics
    weekly_demand_mean = part_orders['Quantity_Ordered'].mean() * 0.25  # Convert to weekly
    weekly_demand_std = part_orders['Quantity_Ordered'].std() * 0.25

    # Calculate lead time characteristics
    avg_lead_time = part_orders['Actual_Lead_Time'].mean() / 7  # Convert to weeks
    lead_time_std = part_orders['Actual_Lead_Time'].std() / 7

    # Service level targets based on criticality
    service_levels = {'A': 0.98, 'B': 0.95, 'C': 0.90}
    target_service_level = service_levels[part_info['Criticality']]

    # Z-score for service level
    z_scores = {0.90: 1.28, 0.95: 1.65, 0.98: 2.05}
    z_score = z_scores[target_service_level]

    # Safety stock calculation (standard formula)
    demand_variability = weekly_demand_std * np.sqrt(avg_lead_time)
    lead_time_variability = weekly_demand_mean * lead_time_std

    total_variability = np.sqrt(demand_variability**2 + lead_time_variability**2)
    safety_stock = z_score * total_variability

    # Calculate current vs recommended difference
    current_ss = current_stock if current_stock else safety_stock  # Default assumption
    ss_change = safety_stock - current_ss

    # Cost implications
    unit_price = part_info['Unit_Price_INR']
    carrying_cost_rate = 0.25  # 25% annual carrying cost
    carrying_cost_change = ss_change * unit_price * carrying_cost_rate

    # Stockout cost estimate (based on production impact)
    criticality_impact = {'A': 100000, 'B': 50000, 'C': 20000}  # Daily production loss
    expected_stockout_cost = criticality_impact[part_info['Criticality']] * 0.02  # 2% chance

    return {
        'Part_ID': part_id,
        'Part_Name': part_info['Part_Name'],
        'Criticality': part_info['Criticality'],
        'Current_Safety_Stock': current_ss,
        'Recommended_Safety_Stock': max(1, round(safety_stock)),
        'Safety_Stock_Change': round(ss_change),
        'Weekly_Demand_Mean': round(weekly_demand_mean, 1),
        'Lead_Time_Weeks': round(avg_lead_time, 1),
        'Target_Service_Level': target_service_level,
        'Annual_Carrying_Cost_Change': round(carrying_cost_change),
        'Expected_Stockout_Cost_Reduction': round(expected_stockout_cost)
    }

# Calculate safety stock recommendations
safety_stock_recommendations = []
for part_id in parts_df['Part_ID'].head(8):  # Top 8 parts for demo
    ss_rec = calculate_optimal_safety_stock(part_id)
    if ss_rec:
        safety_stock_recommendations.append(ss_rec)

safety_stock_df = pd.DataFrame(safety_stock_recommendations)

print("✅ Safety stock optimization completed")
print(f"\n📦 Safety Stock Recommendations (Top 5):")
safety_stock_sorted = safety_stock_df.sort_values('Safety_Stock_Change', ascending=False)
for idx, row in safety_stock_sorted.head().iterrows():
    change_direction = "Increase" if row['Safety_Stock_Change'] > 0 else "Decrease"
    print(f"   {idx+1}. {row['Part_Name']} ({row['Criticality']}):")
    print(f"      {change_direction} by {abs(row['Safety_Stock_Change']):.0f} units")
    print(f"      Target Service Level: {row['Target_Service_Level']:.0%}")

# Supplier Diversification Recommendations
print(f"\n🔄 Supplier Diversification Analysis...")

diversification_recommendations = []

for _, part in part_risk_df.iterrows():
    if part['Supplier_Count'] < 2 and part['Criticality'] == 'A':
        # Critical parts with single supplier = high risk
        primary_supplier_id = enhanced_df[enhanced_df['Part_ID'] == part['Part_ID']]['Supplier_ID'].iloc[0]
        primary_supplier_risk = supplier_risk_final_df[supplier_risk_final_df['Supplier_ID'] == primary_supplier_id]['Risk_Score'].iloc[0]

        # Find alternative suppliers (with lower risk scores)
        alternative_suppliers = supplier_risk_final_df[
            (supplier_risk_final_df['Risk_Score'] < primary_supplier_risk) &
            (supplier_risk_final_df['Supplier_ID'] != primary_supplier_id)
        ].head(2)

        diversification_recommendations.append({
            'Part_ID': part['Part_ID'],
            'Part_Name': part['Part_Name'],
            'Current_Supplier_Count': part['Supplier_Count'],
            'Primary_Supplier_Risk': primary_supplier_risk,
            'Recommended_Action': 'Dual Source',
            'Alternative_Suppliers': ', '.join(alternative_suppliers['Supplier_Name'].tolist()),
            'Risk_Reduction_Potential': primary_supplier_risk * 0.3  # 30% risk reduction
        })

diversification_df = pd.DataFrame(diversification_recommendations)

print("✅ Supplier diversification analysis completed")
if len(diversification_df) > 0:
    print(f"\n🔄 Dual Sourcing Recommendations:")
    for idx, row in diversification_df.iterrows():
        print(f"   {idx+1}. {row['Part_Name']}: Add {row['Alternative_Suppliers']}")
        print(f"      Risk Reduction: {row['Risk_Reduction_Potential']:.1f} points")

# Generate Business Intelligence Export Files
print(f"\n📤 Generating BI Export Files...")

# 1. Supplier Risk Scorecard (for BI dashboard)
supplier_scorecard = supplier_risk_final_df[[
    'Supplier_ID', 'Supplier_Name', 'Location', 'Supplier_Type',
    'OTD_Performance', 'Avg_Delay_Days', 'Risk_Score', 'Risk_Category',
    'Total_Orders', 'Total_Order_Value'
]].copy()

supplier_scorecard['OTD_Performance'] = (supplier_scorecard['OTD_Performance'] * 100).round(1)
supplier_scorecard['Risk_Score'] = supplier_scorecard['Risk_Score'].round(1)
supplier_scorecard['Total_Order_Value'] = supplier_scorecard['Total_Order_Value'].round(0)

# 2. Part Risk Analysis (for BI dashboard)
part_scorecard = part_risk_df[[
    'Part_ID', 'Part_Name', 'Category', 'Criticality',
    'Supplier_Count', 'OTD_Performance', 'Part_Risk_Score', 'Risk_Category',
    'Total_Order_Value'
]].copy()

part_scorecard['OTD_Performance'] = (part_scorecard['OTD_Performance'] * 100).round(1)
part_scorecard['Part_Risk_Score'] = part_scorecard['Part_Risk_Score'].round(1)

# 3. PO Risk Predictions (for operational dashboard)
po_predictions = enhanced_df[[
    'PO_Number', 'PO_Date', 'Part_ID', 'Supplier_ID',
    'Promised_Delivery_Date', 'Actual_Delivery_Date',
    'Delay_Probability_RF', 'Delay_Days', 'Order_Value_INR', 'Is_Late'
]].copy()

po_predictions['Delay_Probability_RF'] = (po_predictions['Delay_Probability_RF'] * 100).round(1)
po_predictions['Risk_Level'] = pd.cut(po_predictions['Delay_Probability_RF'],
                                     bins=[0, 25, 50, 75, 100],
                                     labels=['Low', 'Medium', 'High', 'Critical'])

# 4. Fill Rate Simulation Results (for scenario analysis)
fill_rate_forecast = simulation_results_df.copy()
fill_rate_forecast['On_Time_Rate'] = (fill_rate_forecast['On_Time_Rate'] * 100).round(1)
fill_rate_forecast['Stockout_Risk'] = (fill_rate_forecast['Stockout_Risk'] * 100).round(1)

# 5. Optimization Recommendations Summary
optimization_summary = {
    'Safety_Stock_Recommendations': len(safety_stock_df),
    'Total_Carrying_Cost_Change': safety_stock_df['Annual_Carrying_Cost_Change'].sum(),
    'Parts_Needing_Dual_Sourcing': len(diversification_df),
    'High_Risk_Suppliers': len(supplier_scorecard[supplier_scorecard['Risk_Category'].isin(['High Risk', 'Critical Risk'])]),
    'High_Risk_Parts': len(part_scorecard[part_scorecard['Risk_Category'] == 'High Risk']),
    'Average_Fill_Rate': fill_rate_forecast['On_Time_Rate'].mean(),
    'Generation_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

print("✅ BI export files prepared")

# Summary Report
print(f"\n📊 PROJECT SUMMARY REPORT")
print("=" * 50)

print(f"🎯 KEY FINDINGS:")
print(f"   • Analyzed {len(enhanced_df)} purchase orders from {len(suppliers_df)} suppliers")
print(f"   • Overall OTD Performance: {enhanced_df['Is_On_Time'].mean():.1%}")
print(f"   • {len(supplier_scorecard[supplier_scorecard['Risk_Category'].isin(['High Risk', 'Critical Risk'])])} suppliers classified as high/critical risk")
print(f"   • {len(part_scorecard[part_scorecard['Risk_Category'] == 'High Risk'])} parts identified as high-risk")
print(f"   • Predicted fill rate: {optimization_summary['Average_Fill_Rate']:.1f}%")

print(f"\n💡 OPTIMIZATION RECOMMENDATIONS:")
print(f"   • Adjust safety stock for {len(safety_stock_df)} critical parts")
print(f"   • Implement dual sourcing for {len(diversification_df)} single-source critical parts")
print(f"   • Focus supplier improvement efforts on {supplier_scorecard[supplier_scorecard['Risk_Score'] > 30].shape[0]} medium/high-risk suppliers")
print(f"   • Estimated annual carrying cost impact: ₹{abs(optimization_summary['Total_Carrying_Cost_Change']):,.0f}")

print(f"\n📈 NEXT STEPS:")
print("   1. Deploy predictive models in production environment")
print("   2. Implement automated risk scoring dashboard")
print("   3. Establish supplier performance monitoring")
print("   4. Execute safety stock optimization plan")
print("   5. Initiate dual sourcing negotiations for critical parts")

print(f"\n✅ Python Analytics Pipeline Completed Successfully!")
print(f"   Ready for BI Dashboard Development and Stakeholder Presentation")


🎯 Step 8: Optimization Recommendations & BI Outputs

💡 Why Optimization Recommendations?
Optimization provides:
- Actionable strategies to reduce supply risk
- Cost-benefit analysis of mitigation options
- Prioritized list of interventions
- ROI estimates for risk reduction initiatives

📦 Safety Stock Optimization Analysis...
✅ Safety stock optimization completed

📦 Safety Stock Recommendations (Top 5):
   1. Hydraulic Pump (A):
      Decrease by 0 units
      Target Service Level: 98%
   2. Hydraulic Cylinder (A):
      Decrease by 0 units
      Target Service Level: 98%
   3. Hydraulic Filter (A):
      Decrease by 0 units
      Target Service Level: 98%
   4. Hydraulic Hose (A):
      Decrease by 0 units
      Target Service Level: 98%
   5. Control Valve (A):
      Decrease by 0 units
      Target Service Level: 98%

🔄 Supplier Diversification Analysis...
✅ Supplier diversification analysis completed

📤 Generating BI Export Files...
✅ BI export files prepared

📊 PROJECT SUMMARY RE

In [ ]:
# Final Step: Export Analysis Results for BI Dashboard
print("\n📤 FINAL STEP: Exporting Analysis Results for BI Dashboard")
print("=" * 60)

# Save key analysis results to CSV files for Power BI consumption
import os

# Create outputs directory
if not os.path.exists('bi_outputs'):
    os.makedirs('bi_outputs')

print("🔄 Exporting analysis results...")

# 1. Supplier Risk Scorecard
supplier_scorecard.to_csv('bi_outputs/supplier_risk_scorecard.csv', index=False)
print("✅ supplier_risk_scorecard.csv - Comprehensive supplier performance and risk metrics")

# 2. Part Risk Analysis
part_scorecard.to_csv('bi_outputs/part_risk_analysis.csv', index=False)
print("✅ part_risk_analysis.csv - Part-level risk scores and exposure analysis")

# 3. Purchase Order Predictions
po_predictions.to_csv('bi_outputs/po_risk_predictions.csv', index=False)
print("✅ po_risk_predictions.csv - Individual PO risk scores and delay predictions")

# 4. Fill Rate Simulation Results
fill_rate_forecast.to_csv('bi_outputs/fill_rate_simulation.csv', index=False)
print("✅ fill_rate_simulation.csv - Monte Carlo simulation results for production planning")

# 5. Safety Stock Recommendations
if len(safety_stock_df) > 0:
    safety_stock_df.to_csv('bi_outputs/safety_stock_recommendations.csv', index=False)
    print("✅ safety_stock_recommendations.csv - Optimized inventory level recommendations")

# 6. Feature Importance for Model Explainability
feature_importance.to_csv('bi_outputs/model_feature_importance.csv', index=False)
print("✅ model_feature_importance.csv - Key drivers of delivery delay risk")

# 7. Summary KPIs for Executive Dashboard
summary_kpis = pd.DataFrame([{
    'Metric': 'Total Purchase Orders Analyzed',
    'Value': len(enhanced_df),
    'Format': 'Number'
}, {
    'Metric': 'Overall On-Time Delivery Rate',
    'Value': round(enhanced_df['Is_On_Time'].mean() * 100, 1),
    'Format': 'Percentage'
}, {
    'Metric': 'Average Delay for Late Deliveries',
    'Value': round(enhanced_df[enhanced_df['Is_Late']]['Delay_Days'].mean(), 1),
    'Format': 'Days'
}, {
    'Metric': 'Suppliers with High Risk Rating',
    'Value': len(supplier_scorecard[supplier_scorecard['Risk_Score'] > 50]),
    'Format': 'Number'
}, {
    'Metric': 'Parts with High Risk Rating',
    'Value': len(part_scorecard[part_scorecard['Risk_Category'] == 'High Risk']),
    'Format': 'Number'
}, {
    'Metric': 'Predicted Fill Rate',
    'Value': round(fill_rate_forecast['On_Time_Rate'].mean(), 1),
    'Format': 'Percentage'
}, {
    'Metric': 'Total Order Value Analyzed',
    'Value': round(enhanced_df['Order_Value_INR'].sum() / 1000000, 1),
    'Format': 'Million INR'
}])

summary_kpis.to_csv('bi_outputs/executive_summary_kpis.csv', index=False)
print("✅ executive_summary_kpis.csv - Key performance indicators for executive dashboard")

print(f"\n📊 All analysis results exported to 'bi_outputs/' directory")

# COMPREHENSIVE PROJECT METHODOLOGY SUMMARY
print(f"\n🎓 COMPLETE PROJECT METHODOLOGY SUMMARY")
print("=" * 50)

methodology_summary = f"""
EXCAVATOR SUPPLIER RISK ANALYSIS PROJECT - PYTHON IMPLEMENTATION

📋 PROJECT SCOPE:
• Analyzed supply chain risk for excavator spare parts manufacturing
• Built predictive models for delivery delay risk assessment
• Implemented Monte Carlo simulation for fill rate impact analysis
• Generated optimization recommendations for risk mitigation

🔧 TECHNICAL IMPLEMENTATION:

1. DATA PREPARATION & FEATURE ENGINEERING ({len(enhanced_df.columns)} features)
   • Merged purchase orders with supplier and part master data
   • Created rolling performance metrics (30-day, 90-day supplier OTD rates)
   • Generated temporal features (seasonality, day-of-week effects)
   • Calculated risk scores for parts and suppliers
   • Engineered order complexity and criticality indicators

2. PREDICTIVE MODELING (AUC: {auc_score:.3f})
   • Random Forest Classifier for binary delay prediction
   • Logistic Regression for interpretable risk factor analysis
   • Feature importance analysis identifying top delay drivers
   • Model validation using train-test split methodology

3. RISK SCORING SYSTEM (0-100 scale)
   • Composite supplier risk scores (Performance + Volatility + Strategic + Predictive)
   • Part-level risk assessment based on criticality and supplier diversity
   • Automated risk categorization (Low/Medium/High/Critical)

4. MONTE CARLO SIMULATION ({num_simulations} scenarios per analysis)
   • Probabilistic modeling of delivery performance uncertainty
   • Production fill rate impact assessment under various scenarios
   • Stockout risk quantification for critical components

5. OPTIMIZATION RECOMMENDATIONS
   • Safety stock optimization using service level targets
   • Supplier diversification analysis for single-source risks
   • Cost-benefit analysis of risk mitigation strategies

📊 KEY OUTPUTS FOR BI DASHBOARD:
• Supplier Risk Scorecards - Performance metrics and risk ratings
• Part Risk Analysis - Component-level exposure assessment
• PO Risk Predictions - Real-time delivery delay probability
• Fill Rate Simulation - Production planning scenario analysis
• Safety Stock Recommendations - Inventory optimization guidance
• Executive KPIs - Summary metrics for management reporting

🎯 BUSINESS VALUE DELIVERED:
• Proactive risk identification before delays occur
• Data-driven supplier performance management
• Optimized inventory levels balancing cost and service
• Quantified financial impact of supply chain disruptions
• Actionable recommendations for risk mitigation

📈 SCALABILITY & DEPLOYMENT:
• Modular code structure for easy maintenance and updates
• CSV export format compatible with Power BI and other BI tools
• Model artifacts ready for production deployment
• Automated pipeline suitable for regular refresh cycles

🔄 CONTINUOUS IMPROVEMENT:
• Model performance monitoring framework
• Regular retraining with new data
• A/B testing for optimization strategy effectiveness
• Feedback loops for model refinement

Total Processing Time: Real-time capable for interactive dashboards
Data Volume: {len(enhanced_df):,} records processed across {len(suppliers_df)} suppliers
Model Accuracy: {((y_pred_binary == y_test_binary).mean() * 100):.1f}% on test set
Risk Assessment: {len(supplier_scorecard)} suppliers and {len(part_scorecard)} parts analyzed

✅ PROJECT READY FOR BUSINESS INTELLIGENCE DASHBOARD DEVELOPMENT
"""

print(methodology_summary)

# Save methodology summary
with open('bi_outputs/project_methodology_summary.txt', 'w') as f:
    f.write(methodology_summary)

print("✅ Complete project documentation saved")

print(f"\n🚀 EXCAVATOR SUPPLIER RISK ANALYSIS PROJECT COMPLETED!")
print(f"   All Python analytics completed successfully")
print(f"   Ready for Power BI dashboard development")
print(f"   {len(os.listdir('bi_outputs'))} output files generated for BI consumption")


📤 FINAL STEP: Exporting Analysis Results for BI Dashboard
🔄 Exporting analysis results...
✅ supplier_risk_scorecard.csv - Comprehensive supplier performance and risk metrics
✅ part_risk_analysis.csv - Part-level risk scores and exposure analysis
✅ po_risk_predictions.csv - Individual PO risk scores and delay predictions
✅ fill_rate_simulation.csv - Monte Carlo simulation results for production planning
✅ safety_stock_recommendations.csv - Optimized inventory level recommendations
✅ model_feature_importance.csv - Key drivers of delivery delay risk
✅ executive_summary_kpis.csv - Key performance indicators for executive dashboard

📊 All analysis results exported to 'bi_outputs/' directory

🎓 COMPLETE PROJECT METHODOLOGY SUMMARY

EXCAVATOR SUPPLIER RISK ANALYSIS PROJECT - PYTHON IMPLEMENTATION

📋 PROJECT SCOPE:
• Analyzed supply chain risk for excavator spare parts manufacturing
• Built predictive models for delivery delay risk assessment
• Implemented Monte Carlo simulation for fill rate